Keras Classifier


In [36]:
import pandas as pd
from datasets import  load_dataset

import keras
from keras import layers
import tensorflow as tf
from transformers import BertTokenizer, DataCollatorWithPadding
import numpy as np
import keras.backend as K
from sklearn.preprocessing import OneHotEncoder


Tokenizer 

Bert-base-uncased a 110 million paramater corpus which disregards captilisation

In [31]:
dataset = load_dataset("Brecon/Train_Test")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(token):
    return tokenizer(token["text"], truncation=True, padding="max_length", max_length=512)

In [32]:
tokenized_dataset = dataset.map(preprocess_function,batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

In [57]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1


In [34]:
print(tokenized_dataset['train'])

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 122
})


In [58]:
# Create a Keras model for classification

num_classes = 3
classifier_inputs = tf.keras.Input(shape=(512,))
dense_layer = layers.Dense(128, activation='relu')(classifier_inputs)
train_label = []
test_label = []
values = []

for label in tokenized_dataset['train']['label']:
    one_shot_label = [0,0,0]
    one_shot_label[label] = 1
    train_label.append(one_shot_label)

for label in tokenized_dataset['test']['label']:
    one_shot_label = [0,0,0]
    one_shot_label[label] = 1
    test_label.append(one_shot_label)


for _ in range(20):
    # Add 10 additional dense layers
    for _ in range(10):
        dense_layer = layers.Dense(128, activation='relu')(dense_layer)

    # Classification layer
    output_layer = layers.Dense(num_classes, activation='softmax')(dense_layer)

    # Set up the model with classifier
    model = keras.Model(inputs=classifier_inputs, outputs=output_layer)

    # Compile the model with an appropriate loss function and optimizer
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1_score])

    # Convert target values to one-hot encoded arrays
    num_classes = 3  # Set the correct number of classes (3 for your case)
    input_shape = model.layers[0].input_shape

    # Train the model
    model.fit(tokenized_dataset['train']['input_ids'], train_label, batch_size=32, epochs=3)

    # Evaluate model on the test set
    loss,accuracy,f1 = model.evaluate(tokenized_dataset['test']['input_ids'], test_label)
    values.append([f1,accuracy])

Epoch 1/3
4/4 [==============================] - 2s 7ms/step - loss: 146.2339 - accuracy: 0.3689 - f1_score: 0.3660
Epoch 2/3
4/4 [==============================] - 0s 9ms/step - loss: 35.4341 - accuracy: 0.4344 - f1_score: 0.4321
Epoch 3/3
1/1 [==============================] - 0s 395ms/step - loss: 17.8004 - accuracy: 0.4516 - f1_score: 0.4516
Validation Loss: 17.800424575805664
Validation Accuracy: 0.4516128897666931
F1-Score: 0.45161283016204834
Epoch 1/3
4/4 [==============================] - 5s 13ms/step - loss: 1.2812 - accuracy: 0.4262 - f1_score: 0.2679
Epoch 2/3
4/4 [==============================] - 0s 20ms/step - loss: 1.0370 - accuracy: 0.4508 - f1_score: 0.3918
Epoch 3/3
1/1 [==============================] - 1s 509ms/step - loss: 1.0157 - accuracy: 0.3548 - f1_score: 0.3729
Validation Loss: 1.015738606452942
Validation Accuracy: 0.35483869910240173
F1-Score: 0.37288129329681396
Epoch 1/3
4/4 [==============================] - 6s 20ms/step - loss: 1.0060 - accuracy: 0.426

In [61]:
acc_avg = 0
f1_avg = 0
for value in values:
    acc_avg += value[1]
    f1_avg += value[0]

acc_avg = acc_avg/20
f1_avg = f1_avg/20
print(acc_avg)
print(f1_avg)

0.4629032239317894
0.15427086502313614


In [ ]:
master_dataset = load_dataset('Brecon/Master_Train_Test')

new_tokenized_dataset = master_dataset.map(preprocess_function, batched=True)

predictions = model.predict(new_tokenized_dataset['test']['input_ids'])

prediction_array =[]
for i in predictions:
    prediction_array.append(np.argmax(i))



In [70]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, recall_score, precision_score

f1 = f1_score(new_tokenized_dataset['test']['label'], prediction_array, average="weighted")
recall = recall_score(new_tokenized_dataset['test']['label'], prediction_array, average="weighted")
precision = precision_score(new_tokenized_dataset['test']['label'], prediction_array, average="weighted")

print(f1,recall,precision)

0.30326390027882566 0.4725274725274725 0.2232822122932013


c:\Users\breco\anaconda3\envs\comp815\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
